In [1]:
#import everything we'll need
import pandas as pd
import geopandas as gp
import shapely as sp
from shapely.geometry import Point, Polygon, MultiPolygon
import numpy as np
from itertools import combinations
import intake
from shapely.geometry import MultiPoint

In [2]:
#because of update to p4tools, 6/27/19
from p4tools import io

In [3]:
#because of update to p4tools
tiles = io.get_tile_coordinates(update=True)

In [4]:
tiles.head()

,x_tile,y_tile,x_hirise,y_hirise,PlanetocentricLatitude,PlanetographicLatitude,PositiveEast360Longitude,BodyFixedCoordinateX,BodyFixedCoordinateY,BodyFixedCoordinateZ,tile_id,obsid
0,1,1,420.0,324.0,-85.504144,-85.556701,104.952104,-68.380674,256.056147,-3370.636698,APF0000cwz,ESP_012079_0945
1,1,2,420.0,872.0,-85.502300,-85.554878,104.934805,-68.331337,256.181661,-3370.628630,APF0000ck9,ESP_012079_0945
2,1,3,420.0,1420.0,-85.500454,-85.553054,104.917486,-68.281853,256.307249,-3370.620295,APF0000cty,ESP_012079_0945
3,1,4,420.0,1968.0,-85.498607,-85.551229,104.900198,-68.232522,256.433112,-3370.615690,APF0000ciy,ESP_012079_0945
4,1,5,420.0,2516.0,-85.496762,-85.549404,104.882968,-68.183281,256.558509,-3370.606455,APF0000cwp,ESP_012079_0945


In [5]:
tiles[tiles['obsid'] == 'ESP_012884_0935' ].head()
#check the tile_coordinates csv, see no X,Y,Z bodyfixed coords.

,x_tile,y_tile,x_hirise,y_hirise,PlanetocentricLatitude,PlanetographicLatitude,PositiveEast360Longitude,BodyFixedCoordinateX,BodyFixedCoordinateY,BodyFixedCoordinateZ,tile_id,obsid
4777,1,1,420.0,324.0,-86.455091,-86.496595,NaN,NaN,NaN,NaN,APF0000dsd,ESP_012884_0935
4778,1,2,420.0,872.0,-86.452087,-86.493626,NaN,NaN,NaN,NaN,APF0000dqa,ESP_012884_0935
4779,1,3,420.0,1420.0,-86.449083,-86.490657,NaN,NaN,NaN,NaN,APF0000dru,ESP_012884_0935
4780,1,4,420.0,1968.0,-86.446077,-86.487686,NaN,NaN,NaN,NaN,APF0000dpx,ESP_012884_0935
4781,1,5,420.0,2516.0,-86.443066,-86.484709,NaN,NaN,NaN,NaN,APF0000dsb,ESP_012884_0935


In [ ]:
#michael's code, gives the polygon for a given obsid
def get_outer_polygon_for_obsid(obsid, return_hull=True):
    #tiles = pd.read_csv("tile_coordinates.csv")
    #replace above line with something that takes update
    #tiles = io.get_tile_coordinates(update=True)
    obsid_tiles = tiles.query("obsid == @obsid")
    cols = "BodyFixedCoordinateX BodyFixedCoordinateY".split()
    coords = obsid_tiles[cols]
    coords.columns = ["x", "y"]
    mp = MultiPoint(list(zip(coords.x, coords.y)))
    if return_hull:
        return mp.convex_hull
    else:
        return mp

In [ ]:
#my code, goes through each possible combination of polygons and gives areas
def interchecker(polygon_array):
    """
    
    """
    bucket2 = []
    combos = []
    iterable1 = range(len(polygon_array)) 
    p = len(polygon_array)
    for n in iterable1:
        if p > 0:
            comb = list(combinations(iterable1,p))
            for item in comb:
                x = item
                combos.append(x)
            p = p-1
        else:
            end

    #the above gives us an array of each possible combination eg
    #combos = [comb[0],comb[1],...]
    #where comb[0] could be (0,1,2,4) or something of the sort
    
    for ele in combos:
        d = {}
        #print(ele) #shows the element used
        #d['region_name'] = region
        #d['index_used'] = ele #for testing
        bucket = []
        for number in ele:
            bucket.append(polygon_array[number])
            #bucket now [poly0, poly1, poly2, poly4] if using comb[0] above
        d['elements_used'] = np.array(bucket)
        if len(bucket)>1:
            y = bucket[0].intersection(bucket[1])
            for i in range(len(bucket)-2):
                y = y.intersection(bucket[i+2])
            #print(y.area)
            d["combo_area"]= y.area
        else:
            #print(bucket[0].area)
            d['combo_area'] = bucket[0].area
        #for number in ele:
            #d['elements_used'] = polygon_array[number]
            #must move elements used to get every element used- where?
        bucket2.append(d)
    return pd.DataFrame(bucket2)

In [ ]:
olddata = pd.read_csv("n_obj_df.csv") 
#this df is created in calculate_fractional_coverage_new if it needs changing.

In [ ]:
olddata.head() #check it loaded in correctly

In [ ]:
allregions = olddata.region_name.unique() #get all regions

In [ ]:
allregions #check all regions

In [ ]:
#testing new stuff from michael, 6/27/19
from p4tools import io
fans = io.get_fan_catalog(update=True)

In [ ]:
fans[fans['obsid'] == 'ESP_012884_0935'].head()
#can see they all have bodyfixed x, y, z coords now.  Need to fix olddata!

In [ ]:
test = olddata[olddata['region_name'] == 'Manhattan_Classic']
#this is where it was broken before

In [ ]:
get_outer_polygon_for_obsid('ESP_012884_0935')
#this should give a polygon, it doesn't

In [ ]:
tiles[tiles['obsid'] == 'ESP_012884_0935' ].head()
#check the tile_coordinates csv, see no X,Y,Z bodyfixed coords.

Can see for these tiles there's no x, y, z coords.  Am I on an old version of this or is this another broken df?  Figure out how to update to michael's current version, and if this is it let him know it's still broken.  I have planetfour-catalog v 1.1.2, ask him how current that is.

In [ ]:
for region in allregions:
    bucket = []
    for obs in olddata[olddata['region_name'] == region].obsid.unique():
        d = {}
        d['region_name'] = region
        d['Obsid_Used'] = obs
        x = get_outer_polygon_for_obsid(obs)
        area = x.area
        d['Obsid_Area'] = area
        d['polygon_shape'] = x
        bucket.append(d)
    results = pd.DataFrame(bucket)
    
    bucket = []
    for obs in olddata[olddata['region_name'] == region].obsid.unique():
        d = {}
        d['obsid'] = obs #we dont want obs, want obs used in combo
        x = get_outer_polygon_for_obsid(obs)
        d['obs_poly'] = x
        bucket.append(d)
    testresult = pd.DataFrame(bucket)
    
    df1 = testresult
    print(testresult)
    print(testresult.obs_poly)
    df2 = interchecker(testresult.obs_poly)
    
    Testregion_inter = interchecker(results.polygon_shape) #all combos of results
    
    bucket2 = []
    for ele in Testregion_inter.index:
        polylist = Testregion_inter.elements_used[ele]
        bucket = []
        for poly in polylist:
            x = df1.obsid[df1['obs_poly'] == poly].values[0]
            bucket.append(x)
        bucket = tuple(bucket)
        bucket2.append(bucket)
    Testregion_inter['obsids_used'] = np.array(bucket2)

    obsdf = Testregion_inter
    
    bucket = []
    for i in range(len(obsdf.elements_used)): #change to range(len(obsdf.elements_used)) after we check
        #print(len(obsdf.elements_used[i])) #check it's what we want.
        obsdf.at[i, 'number_hirise'] = int(len(obsdf.elements_used[i]))
        #even calling it int(len) still gives float on df, but whatever.
        
        
    bucket2 = []
    for obslist in obsdf.obsids_used:
        bucket = []
        for obs in obslist:
            x = olddata.Ls[olddata['obsid'] == obs].values[0]
            bucket.append(x)
        bucket = tuple(bucket)
        bucket2.append(bucket)
    obsdf['Ls_used'] = np.array(bucket2)
    obsdf['region_name'] = region

    obsdf.to_csv(f'{region}_duplicate_test_combodf.csv', index = False)